In [5]:
import pandas as pd
import logging
import boto3
from datetime import datetime
import io
from botocore.config import Config

today = datetime.now().strftime('%Y%m%d')
current_ts = datetime.now().strftime('%Y%m%d%H%M%S')

s3_client = boto3.client('s3', endpoint_url='http://localhost:9000')
s3_res = boto3.resource('s3', endpoint_url='http://localhost:9000')

In [6]:
def save_df_to_s3(df, bucket, key, index=False):
    ''' Store df as a buffer, then save buffer to s3'''
    # s3_res = get_s3_resource()
    try:
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=index)
        s3_res.Object(bucket, key).put(Body=csv_buffer.getvalue())
        logging.info(f'{key} saved to s3 bucket {bucket}')
    except Exception as e:
        raise logging.exception(e)

In [ ]:
from airflow import DAG
#from pandas_object_storage.read_write_object_str import load_files_to_object, transform_save_object_silver
from datetime import datetime, timedelta

from airflow.operators.python import PythonOperator

start_date = datetime(2022, 10, 11)
current_ts = datetime.now().strftime('%Y%m%d_%H%M%S')
default_args = {
    'owner': 'train',
    'start_date': start_date,
    'retries': 1,
    'retry_delay': timedelta(seconds=5)
}

with DAG('pull_data', default_args=default_args, schedule_interval='@hourly', catchup=False) as dag:
    pull_data = PythonOperator(task_id='pull_data', python_callable=save_df_to_s3)